<a href="https://colab.research.google.com/github/RafaelCaballero/BME/blob/main/c%C3%B3digo/12estadisticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
### Rafa Caballero

### Estadísticas básicas

### Índice
1. [Introducción](#Introducción)<br>
2. [Centralidad](#Centralidad)<br>
3. [Dispersión](#Dispersión)<br>
4. [Representaciones gráficas](#Gráficas)<br>
   4.1 [Histogramas](#Histogramas)<br>
   4.1 [Líneas y Puntos](#Líneas)<br>
   4.2 [Asimetría](#Asimetría)<br>
   4.3 [Curtosis](#Curtosis)<br>
   4.4 [Boxplots](#Boxplots)<br>
   
5. [Outliers](#Outliers)
6. [Test estadísticos](#Tests)<br>
   6.1 [Test de la normal](#Normalidad)<br>
   6.2 [Igualdad de medias](#Vodka)<br>
   6.3  [Bootstrapping](#Bootstrapping)<br>

[Referencias](#Referencias)




<a name="Introducción"></a>
## Introducción

Es lo primero que debemos hacer: conocer los datos, y para esto nos va a servir

- Tomar muestras (`df.sample(100)`), mirar a los "datos" directamente
- `df.info()` para tipos
- `df.describe()` para información numérica rápida
- `df.A.mean()`, `df.A.median()` media, mediana, etc.
- Diagramas: de barras, d líneas, histogramas y los que vengan bien a nuestro problema
- Test y técnicas estadísticas para asegurar la validez de los resultados

Empezando cargando algunas librerías. Tras ejecutar esta casilla ir a "Entorno de ejecución" y dar "Reiniciar Entorno" para que se carguen.


In [ ]:
modules = ["ydata-profiling","sweetviz","PyQt6","plotly","seaborn","fastf1"]

import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
    print("Instalando módulos")
    for m in modules:
        # para el import quitamos [...] y ==...
        p = m.find("[")
        mi = m if p==-1 else m[:p]
        p = mi.find("==")
        mi = mi if p==-1 else mi[:p]
        torch_loader = importlib.util.find_spec(mi)
        if torch_loader is not None:
            print(m," encontrado")
        else:
            print(m," No encontrado, instalando...",end="")
            try:
                r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
                print("¡hecho!")
            except:
                print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

    print("¡Terminado!")

instala(modules)

In [ ]:
# datos de ejemplo
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/refs/heads/master/datos/madrid/contaminacionLargo.csv"
df = pd.read_csv(url,parse_dates=['fechaH','fecha'])
df

In [ ]:
df.info() # información básica de tipos

In [ ]:
df.describe(include='all') # información estadística básica

In [ ]:
# para conocer el número de valores por columna
for c in df.columns:
  print(c, len(df[c].dropna().unique()))

In [ ]:
# para los valores discretos que no se repiten demasiado puede interesar ver qué valores concretos y qué frecuencia
df.ANO.value_counts()

Queremos que "ANO" sea de tipo entero, no número real. Aprovechamos para poner "AÑO"

In [ ]:

import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/refs/heads/master/datos/madrid/contaminacionLargo.csv"
df_conta = pd.read_csv(url,parse_dates=['fechaH','fecha'])
df_conta.index = df_conta.fechaH
df_conta["AÑO"] = df_conta.ANO.astype('Int64') # nombre raro de los enteros
df_conta = df_conta.drop(columns=["ANO"])
df_conta["AÑO"].value_counts()

**Ejercicio** Comprobar qué valores diferentes toma la columan festivo y la frecuencia de cada uno en `df_conta`

Aunque aquí vamos a ir viendo en detalle cada posibilidad de gráficas y estadísticas se puede obtener una gran cantidad de información en un informe con una librería adecuada; es un excelente punto de comienzo.

In [ ]:
# una librería para tener mucha información de golpe. Muy útil para empezar
from ydata_profiling import ProfileReport
profile = ProfileReport(df)
profile.to_notebook_iframe()
profile.to_file("contamina.html")

Otra posibilidad

In [ ]:
import sweetviz as sv
report = sv.analyze(df)
report.show_html()

<a name="Centralidad"></a>
## Centralidad

La idea de las medidas de centralidad es reducir toda la columna a un solo número un "centro", el que mejor la represente. En la práctica hay más de una posibilidad.


* Moda: el valor que más se repite. La única medida que tiene sentido para variables categóricas.

* Media: $\mu(x) = \frac{\displaystyle {\sum_{i=1}^{N} x_i}}{N}$, donde $x$ es la variable que estamos estudiando formada por $x_1, \dots, x_N$. La media es la medida de centralidad más popular. Puede verse afectada si hay hay una proporción grande de valores demasiado grandes o pequeños (outliers), especialmente si estos están agrupados de forma desigual.

* Mediana: valor que deja al 50% de los valores por debajo y el otro 50% por encima.

Para ver qué medida de centralidad es aplicable en cada caso tenemos que considerar el tipo de variable y si es discreta o continua:

| **Tipo de Variable**     | **Nominal**                  | **Ordinal**                  | **Intervalo**                | **Ratio**                    |
|--------------------------|------------------------------|------------------------------|------------------------------|------------------------------|
| **Discreta**             | Moda                         | Moda, Mediana                | Moda, Mediana, Media?         | Moda, Mediana, Media?       |
| **Continua**             | No aplica                    | Moda, Mediana                | Moda, Mediana, Media         | Moda, Mediana, Media        
Esta tabla es una buena referencia, pero después hay que mirar cada caso.

**Ejemplo 1** En un valor bursátil que normalmente sube salvo pequeñas bajadas ¿tiene sentido hablar de la moda?

**Ejemplo 2** Tenemos un dataframes con datos de tweets. Una columna que indica la hora a la que se produce el tweet. En nuestro caso casi todos se producen por la noche, más o menos la mitad a las 23h y la mitad a las 0h. ¿Cuál será la media? ¿tiene sentido?

Para obtener estos resultados con Pandas podemos utilizar los métodos predefinidos `mode`, `median` y `mean`.




Ejemplo: notas obtenidas por diferentes países en las pruebas Pisa en lectura (REA), matemáticas (MAT) y ciencias (SCI) tanto para mujeres (FE) como para hombres (MA). Incluye también la renta per capita (RPC) del país y el nombre (PAIS) del país.

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/PisaDataClean.csv"
df_pisa = pd.read_csv(url)
df_pisa

In [ ]:
df_pisa.info()

In [ ]:
df_pisa.describe()

Veamos datos numéricos resaltando los valores extremos:

In [ ]:
df_pisa.style.background_gradient("RdYlGn")

In [ ]:
df_pisa.info() # información general, nulos, tipos y memoria que ocupa

In [ ]:

desc_pisa = df_pisa.describe()
desc_pisa

In [ ]:
# Nos quedamos solo con media y mediana
mediamediana = desc_pisa.loc[ ["mean","50%"] ]
mediamediana

In [ ]:
# veamos la diferecia
mediamediana.iloc[0,:]-mediamediana.iloc[1,:]

In [ ]:
traspuesta = mediamediana.T
traspuesta

In [ ]:
traspuesta["dif"] = traspuesta["mean"] - traspuesta["50%"]
df_dif = traspuesta.T
df_dif

In [ ]:
# otra forma, solo para una variable
df_pisa.MAT.mean(), df_pisa.MAT.median()

**Ejercicio** Partimos del siguiente fichero de valores con incrementos con respecto al día anterior en tanto sobre 1

In [ ]:
import pandas as pd
url = "https://github.com/RafaelCaballero/tdm/raw/refs/heads/master/datos/IBEX6000Inc.zip"

# solución
df_bolsa = pd.read_csv(url,parse_dates=['Fecha'])

# Pasamos a % en lugar de sobre 1
valores = ['BBVA', 'Iberdrola', 'Inditex', 'Repsol', 'Santander']
df_bolsa[valores] = df_bolsa[valores]*100
df_bolsa

Queremos ver qué incremento medio tiene  el BBVA los Jueves y que incremento los Viernes. Mostrar ambos valores

**Ejercicio** Ahora lo mismo pero para cuando Luna es Nueva y cuando Luna es Llena

<a name="Dispersión"></a>
## Dispersión

Si las medidas de centralidad dan la idea de un "centro" de la variable, la media de dispersión sería el "radio" indica lo alejados que están de ese centro. Vamos a ver 2 cada uno relacionado con una de las medidas de centralidad

* Desviación típica $\sigma(x)=\sqrt{\frac{{\displaystyle \sum_{i=1}^{N}\left(x_{i}-\mu\right)^{2}}}{N}}$, la raíz cuadrada de la varianza

* Desviación absoluta con respecto a la mediana $\mathit{MAD}(x) = mediana(|x_i - mediana(x)|)$

In [ ]:
df_pisa.MAT.std(), (df_pisa.MAT - df_pisa.MAT.median()).abs().median()

In [ ]:
def mad(variable):
  return (variable-variable.median()).abs().median()

def centra_disp(df):
  # nos quedamos solo con los datos que sean números (asumimos que hemos comprobado que todos son ratio o intervalo)
  df_num = df.select_dtypes(include=["number"])
  datos = []
  for c in df_num.columns:
      variable = df[c]
      datos.append([variable.mean(), variable.median(), variable.std(), mad(variable)])

  estad = pd.DataFrame(datos,columns=["mean","median","std","MAD"],index=df_num.columns)
  return estad

In [ ]:
centra_disp(df_pisa)

Algunas consecuencias sencillas:
* De media parece que los hombres lo hacen mejor en matemáticas y las mujeres en lectura, en ciencia la diferencia es muy pequeña
* De media se obtiene mejor nota en ciencias que en lectura, y en lectura que en matemáticas
* En general la mediana es mayor que la media indicando una mayor dispersión a la izquierda
* La mayor dispersión `std` se da en MAT_MA, pero si nos fijamos en la mediana es SCI_MA (quizás MAT_MA tiene más outliers?)
* La menor dispersión `std` se da en REA_FE y en SCI_FE, aunque desde el punto de vista de MAD se da en SCI_FE y MAT_FE. En todo caso parece que las notas para las chicas varían menos de país en país que en el caso de los chicos (¿por qué?)

<a name="Gráficas"></a>
## Representaciones gráficas

<a name="Histogramas"></a>
### Histogramas


* Un histograma representa la frecuencia (número de elementos) en una variable (ratio u intervalo) representada por intervalos, nos permite ver la distribución de la variable (algo que no se pretende con el diagrama de barras)

* En el caso de valores ordinales se puede usar un "diagrama de barras ordenado"

Muestran la frecuencia (número de repeticiones) de cada valor.

- Histograma: para variables continuas. Se utilizan intervalos (los bins)
- Diagrama de barras: para variables discretas.

La forma más sencilla


In [ ]:
import matplotlib.pyplot as plt
df_pisa.hist()
plt.tight_layout()
plt.show()

In [ ]:
df_pisa.MAT.plot(kind="hist")

Ejemplo con datos financieros

In [ ]:
import yfinance as yf
import pandas as pd


ticker ='ITX.MC'  #   ITX.MC es Inditex en la bolsa de Madrid

# Descargar los datos de Yahoo Finance
data = yf.download(ticker, start="1900-01-01")

# Seleccionar solo la columna de 'Close' (precios ajustados de cierre)
df = pd.DataFrame({'Close': data['Close']})

# Eliminar las filas con datos perdidos
df = df.dropna()
df



In [ ]:
ax = df.Close.plot(kind="density", figsize=(9,9))
df.Close.plot(kind="hist", density=True, alpha=0.5, bins=30, ax=ax)

Es una bimodal, algo bastante curioso porque representa varios tipos de valores, veamos la gráfica con línea

In [ ]:
df.Close.plot(kind="line")

Otra forma de hacer histogramas

In [ ]:
#!pip install plotly

In [ ]:
import plotly.express as px
fig = px.histogram(df, x="Close")
fig.show()

Histograma clásico un poco más elaborado, con matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#generate a random numpy array with 1000 elements
normaldata = np.random.randn(100000)
data=normaldata

#histograma
plt.hist(data,edgecolor="black", bins =30)

#añadimos título
plt.title("Histograma")

#etiqueta en X
plt.xlabel("Valores")

#etiqueta en y
plt.ylabel("Frecuencias")

# mostrarlo
plt.show()

In [ ]:
data.mean(), data.std(), np.median(data)

Con `density=True` conseguimos frecuencias relativas

In [ ]:
#histograma
plt.hist(data,edgecolor="black", bins =30, density=True)

#añadimos título
plt.title("Histograma")

#etiqueta en X
plt.xlabel("Valores")

#etiqueta en y
plt.ylabel("Frecuencias")

# mostrarlo
plt.show()

Cuando tenemos una normal "perfecta"  se tiene que
<img src="https://news.mit.edu/sites/default/files/styles/news_article__image_gallery/public/images/201202/20120208160239-1_0.jpg?itok=1X1a_HCs" width=400></img>

13.6 + 34.1 + 13.6 + 34.1 = 95.4% es decir casi el 95% de los valores estarán entre la media y +- 2 la desviación típica

In [ ]:
m = data.mean()
s = data.std()

sum((data < m+2*s) & (data >m-2*s))/len(data)

Ejemplo más elaborado usando seaborn y con media y mediana

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

def plot_histograms_with_stats(df):
    # solo columnas numéricas
    columns = df.select_dtypes(include=[np.number]).columns
    for col in columns:
        # nuevo gráfico
        fig,ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
        data = df[col].dropna()
        mean_val = data.mean()
        median_val = data.median()

        # histograma y líneas verticales
        sns.histplot(df[col], bins=25, ax=ax, alpha=0.3, kde=False)
        ax.axvline(mean_val, color='red', linestyle='dashed', linewidth=1, label=f'Mean: {mean_val:.2f}')
        ax.axvline(median_val, color='green', linestyle='dashed', linewidth=1, label=f'Median: {median_val:.2f}')

        ax.set_title(f'Histogram of {col}')
        ax.set_xlabel(col)
        ax.set_ylabel('Frequency')
        ax.legend()
        fig.savefig(col+".png")
        plt.show()
plot_histograms_with_stats(df_pisa)

En el caso de ser strings o tipos categóricos lo que mostramos es el un diagrama de barras  del dataframe devuelto por `value_counts`

**Pandas**

In [ ]:
frecuencias = df_conta['festivo'].value_counts().sort_index()

# Crear un diagrama de barras usando pandas
frecuencias.plot(kind='bar', figsize=(8, 5), color='skyblue', title='Frecuencia de Categorías')
plt.xlabel('Categoría')
plt.ylabel('Frecuencia')
plt.show()

**Seaborn**

In [ ]:
import seaborn as sns
sns.countplot(data=df_conta, x="festivo")

**Plotly**

In [ ]:
fig = px.bar(frecuencias)
fig.show()

<a name="Líneas"></a>
### Diagramas de líneas y puntos

Los diagramas de puntos y los diagramas de líneas son un buen complemento de los histogramas, especialmente cuando son series temporales. Si este es el caso, tener la fecha como índice nos ayudará a que el gráfico se muestre ordenado por fecha.

**Pandas**

In [ ]:
filtro = (df_conta["AÑO"]==2021) & (df_conta.MES<=3)
df_conta[filtro].NOx.plot(kind="line", figsize=(20,5))

El gráfico de puntos es similar, pero exige que se lo apliquemos al dataframe completo e indiquemos como parámetros la x y la y...cosas de Pandas

In [ ]:
df_conta[filtro].plot(kind="scatter", x="fechaH", y="NOx", figsize=(20,5))

In [ ]:
# Añadimos tamaño del punto
df_conta[filtro].plot(kind="scatter", x="fechaH", y="NOx", figsize=(20,5),s=1)

**Seaborn**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 5))
sns.lineplot( data=df_conta[filtro],   x='fechaH',    y='NOx')
plt.show()


**Plotly**

In [ ]:
import plotly.express as px
import pandas as pd


# Crear el gráfico de líneas con Plotly

fig = px.line(df_conta[filtro], x="fechaH",  y='NOx',  width=1000, height=500)

# Añadir detalles al gráfico
fig.update_layout(
    title='Gráfico de Líneas de NOx vs Fecha',
    xaxis_title='Fecha',
    yaxis_title='NOx',
    xaxis=dict(rangeslider=dict(visible=True)),  # Añadir un selector de rango de fecha si es necesario
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()


<a name="Asimetría"></a>
### Asimetría

Asimetría a la derecha: muchos datos acumulados en poco espacio a la izquierda, a la derecha un descenso lento y prolongado

* Se tendrá que media>mediana
* Normalmente solo tendremos que preocuparnos por outliers a la derecha, es decir por valores "excesivamente grandes"
* Sucede por ejemplo en mediciones que por su naturaleza son todas positivas
* Puede tener sentido hacer un estudio diferente a la izquierda y a la derecha de la mediana

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/madrid/contaminacionLargo.csv"
df_conta = pd.read_csv(url)
df_conta

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


data = df_conta.PM10
#histograma
plt.hist(data,edgecolor="black", bins =50)

#añadimos título
plt.title("Histograma")

#etiqueta en X
plt.xlabel("Valores")

#etiqueta en y
plt.ylabel("Frecuencias")

# mostrarlo
plt.show()

In [ ]:
data.mean(), data.median()

Como vemos aquí no se cumple la regla anterior de que en el entorno 2std se concentra el 95% de la población. Esto es así porque no se trata de una normal

In [ ]:
m = data.mean()
s = data.std()

sum((data < m+2*s) & (data >m-2*s))/len(data)

La función `skew` de Pandas nos indica la asimetría:

        >0 : Asimetría a la derecha o positiva
        aprox. 0 : simétrico
        <0 : asimetría a la izquierda o negativa


<img src="https://upload.wikimedia.org/wikipedia/commons/c/cc/Relationship_between_mean_and_median_under_different_skewness.png"  width=500>By Diva Jain - https://codeburst.io/2-important-statistics-terms-you-need-to-know-in-data-science-skewness-and-kurtosis-388fef94eeaa, CC BY-SA 4.0, https://commons.wikimedia.org/w/index.php?curid=84219892</img>





In [ ]:
data.skew()

Tenemos por tanto asimetría a la derecha. En el dataframe de PISA

In [ ]:
df_pisa_num = df_pisa.select_dtypes(include=["number"])
for c in df_pisa_num:
    print(c,df_pisa_num[c].skew())

En nuestros datos "normales"

In [ ]:
pd.DataFrame({"x": normaldata}).skew()

Como vemos no sale exactamente 0 aunque sí muy cercano

<a name="Apuntamiento"></a>
### Curtosis

La curtosis indica el peso de las colas en relación con una normar estándar. A menudo se confunde con "apuntamiento" pero no es exactamente lo mismo. La función `kurtosis` de Pandas nos indica este valor:

        >0 : leptocúrtica ; los outliers tienen más peso que en la normal, tenemos muchos outliers (hay que ver por qué y si merece la pena hacer un estudio solo de esta parte)
        aprox. 0 : ismilar a una normal
        <0 : los outliers tienen menos peso que en la normal, la distribución está más concentrada alrededor de la media



In [ ]:
pd.DataFrame({"x": normaldata}).kurtosis()

In [ ]:
df_conta.PM10.kurtosis()

In [ ]:
df_pisa.MAT.kurtosis()

**Ejemplo**

Datos de 7 sensores de radiación solar durante varios días con todas sus horas

In [ ]:
import pandas as pd
url = "https://github.com/RafaelCaballero/tdm/raw/master/datos/solar.zip"
df_solar = pd.read_csv(url)

In [ ]:
df_solar

In [ ]:

import matplotlib.pyplot as plt
for c in df_solar.columns[4:]:
    fig, ax = plt.subplots(figsize=(24, 6))
    df_solar[c].hist(bins=50)
    plt.title(c)
    plt.show()

¿Qué consecuencias se extraen a simple vista? Añadir código para probar

In [ ]:
df_solar["S1"].skew()

In [ ]:
df_solar["S1"].kurtosis()

<a name="Boxplots"></a>
### Boxplots

También conocidos como diagramas de cajas y bigotes, se basan en la mediana y son muy útiles para detectar outliers.

Ejemplo:

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/RafaelCaballero/tdm/refs/heads/master/datos/f1-Singapur24.csv"
f1 = pd.read_csv(url)
f1

Los boxplots están basados en la mediana y los cuartiles (25% -> Q1, 75% -> Q3) muestran la siguiente información:

<img src="https://miro.medium.com/max/9000/1*2c21SkzJMf3frPXPAR_gZA.png" width=600/>

El criterio para declarar algo como outlier es por tanto ser  $< Q_1 - 1.5 \times (Q_3- Q_1)$  o  bien $> Q_3 + 1.5 \times (Q_3- Q_1)$

Se trata de un criterio bastante exigente porque en el caso de una normal suma más del 99% de la población

<img src="https://miro.medium.com/max/8100/1*NRlqiZGQdsIyAu0KzP7LaQ.png" width=600/>

En Python se pueden mostrar tanto con Pandas, como con Matplotlib, Plotly, como con Seaborn.

**Pandas**


In [ ]:
f1.boxplot(column = "LapTime")

In [ ]:
f1.boxplot(column = "LapTime", by="Driver",  figsize=(15,10))

**matplotlib**

Lo interesante que aporta es la posibilidad de incluir el notch que permite ver si la diferencia de mediana es estadísticamente significativa

In [ ]:
df_vueltas = f1[["LapNumber","LapTime","Driver"]]

df_pilotos = df_vueltas.pivot(index="LapNumber", columns="Driver", values="LapTime")

fig, ax = plt.subplots(figsize=(10, 6),dpi=100)
ax.boxplot(df_pilotos.dropna(),    notch = True,labels=df_pilotos.columns)

Más bonito:

In [ ]:
fig, ax = plt.subplots(figsize=(4, 8),dpi=100)
boxplots = ax.boxplot(df_pilotos.dropna(), notch = True,labels=df_pilotos.columns,
             widths = .7,
             patch_artist=True,
             medianprops = dict(linestyle='-', linewidth=2, color='Yellow'),
             boxprops = dict(linestyle='--', linewidth=2, color='Black', facecolor = 'blue', alpha = .4))
# colores
boxplots['boxes'][0].set_facecolor('darkgreen')
boxplots['boxes'][1].set_facecolor('navy')
boxplots['boxes'][2].set_facecolor('grey')
boxplots['boxes'][3].set_facecolor('navy')

plt.ylabel('Tiempo ms', fontsize = 20)
plt.xlabel('Piloto', fontsize = 20)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.show()

**Plotly**

In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.box(f1, y="LapTime",width=1000, height=500)
fig.show()

**Seaborn**

In [ ]:
import seaborn as sns
sns.boxplot(x=f1["LapTime"])

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(10, 5),dpi=100)
sns.boxplot(x=f1["LapTime"],y=f1["Driver"],palette='Set3')

<a name="Outliers"></a>
### Outliers

Por cierto que las mismas ideas del boxplots se pueden emplear para calcular outliers de forma numérica. La idea del boxplot es que un valor x se considera outlier si

$x< Q_1 - 1.5 \times (Q_3- Q_1)$  o  bien $x> Q_3 + 1.5 \times (Q_3- Q_1)$

Solo nos falta poder calcular $Q_1, Q_3$, pero el método `quantile`de Pandas lo pone muy fácil. Vamos a obtener los outliers para cada piloto:

In [ ]:
df_vueltas = f1[["LapNumber","LapTime","Driver"]]

df_pilotos = df_vueltas.pivot(index="LapNumber", columns="Driver", values="LapTime")
df_pilotos

In [ ]:
import numpy as np
def muestra(driver,ax, x,inferior,superior,c):
    filtro_menor = x < inferior
    filtro_mayor = x > superior
    print(driver)
    print("Límites",round(inferior,3),round(superior,3))
    print("Rápido: ",np.round(x[filtro_menor].values,3))
    print("Lento: ",np.round(x[filtro_mayor].values,3))
    print("*"*100)
    return



#fig, ax = plt.subplots(figsize=(10, 6),dpi=100)
for i, driver in enumerate(df_pilotos.columns):
    x = df_pilotos[driver].dropna()
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    inferior = q1 - 1.5*(q3-q1)
    superior = q3 + 1.5*(q3-q1)
    muestra(driver,ax,x,inferior,superior,driver)

Otro método, el llamado *Hampel X84*  utiliza la mediana y el mad para indicar que los puntos que están separados más de $1.4826 \times \theta\times MAD$ de la mediana son puntos extremos, donde $\theta$ es un número positivo (1,2,3,...) que elegimos nosotros. Por ejemplo para $\theta=4$:

In [ ]:
from scipy.stats import median_abs_deviation

driver = "ALO"
x = df_pilotos[driver].dropna()
mediana = x.median()
MAD = median_abs_deviation(x)

inferior = mediana -  1.4826*4*MAD
superior = mediana + 1.4826*4*MAD
filtro_outliers_sup = df_pilotos[driver] > superior
filtro_outliers_inf = df_pilotos[driver] < inferior

print("Outliers inferiores Alonso (vueltas sorprendentemente rápidas)", np.round(df_pilotos[driver][filtro_outliers_inf],3))
print("Outliers inferiores Alonso (vueltas sorprendentemente lentas)", np.round(df_pilotos[driver][filtro_outliers_sup],3))
print("Límites con  Hampel X84, theta=4, ",inferior,superior)

Ojo porque puede haber también outliers en varias dimensiones.



Consideramos el siguiente dataframe

In [ ]:
d1 = np.random.multivariate_normal(mean = np.array([-.5, 0]),
                               cov = np.array([[1, 0], [0, 1]]), size = 100)
d2 = np.random.multivariate_normal(mean = np.array([15, 10]),
                               cov = np.array([[1, 0.3], [.3, 1]]), size = 100)
outliers = np.array([[0, 10],[0, 9.5]])
df = pd.DataFrame(np.concatenate([d1, d2, outliers], axis = 0), columns = ['Var1', 'Var2'])
df

Aparentamente no hay outliers

In [ ]:
df.boxplot()

¿Seguro que no hay outliers? Veamos ambas columnas conjuntamente

In [ ]:
df.plot(kind='scatter', x='Var1', y='Var2', color='blue')

¿Cómo detectar estos puntos? La idea es que hay que tener una idea de punto "demasiado" alejado del resto, para eso utilizaremos la llamada [distancia de Mahalanobis](https://www.statisticshowto.com/mahalanobis-distance/) que tiene la distancia de un punto a un conjunto de puntos. Esto se hace mediante "envolturas elípticas"


In [ ]:
from sklearn.covariance import EllipticEnvelope
ee = EllipticEnvelope(contamination=.1).fit(df)  # la contaminación es la proporción de outliers, la decidimos nosotros
ee # una "envoltura"

In [ ]:
p = ee.predict(df)
p # +1 normal, -1 outlier

In [ ]:
filtro = p==+1
normales = df[filtro]
outliers = df[~filtro]
fig, ax = plt.subplots(figsize=(10, 6),dpi=100)
ax.scatter(normales.Var1,normales.Var2,color="green")
ax.scatter(outliers.Var1,outliers.Var2,color="red")

<a name="Tests"></a>
## Test Estadísticos



En ciencia de datos no hay casi nunca certezas; al fin y al cabo tenemos unos datos particulares, una "muestra", y por eso algunas conclusiones que saquemos pueden ser meras coincidencias o casualidades. Los tests estadísticos nos indican si una hipótesis se verifica o no con una alta probabilidad.

Cada test va orientado a un problema concreto y plantea una hipótesis,
 llamada *hipótesis nula*.

El resultados del test suele ser un valor p que indica la probabilidad de que se cumpla la hipótesis nula además de otros valores (a menudo un estadístico que depende del test)

Si p<0.05 o p<0.01 (depende del nivel de exigencia) rechazamos la hipótesis nula y damos por válida la contraria, la hipótesis alternativa

Es importante observar que si p>0.05 no "aceptamos" la hipótesis nula, solo decimos que no hemos podido rechazarla (en la práctica a menudo esto se toma como prueba de aceptación aunque no sea correcto).

<a name="Normalidad"></a>
### Test de normalidad

In [ ]:
from scipy.stats import normaltest
from scipy import stats

def normal(data):
    _, p = stats.normaltest(data,nan_policy="omit")
    if p<0.05:
        msg = "Se rechaza H0: no sigue una distribución normal"
    else:
        msg = "No se rechaza H0; no podemos descartar  una distribución normal"
    return msg,round(p,4)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#generate a random numpy array with 100000 elements
normaldata = np.random.randn(100000)

plt.hist(normaldata,edgecolor="black", bins =30, density=True)

normal(normaldata)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def normales(df):
    for c in df.columns:
        fig, ax = plt.subplots(figsize=(5, 3))
        df[c].hist(bins=20)
        msg,p = normal(df[c])
        plt.title(f"{c} {msg} - p: {p} ")
        plt.show()

In [ ]:
import pandas as pd
url = "https://github.com/RafaelCaballero/tdm/raw/refs/heads/master/datos/IBEX6000Inc.zip"

# solución
df_bolsa = pd.read_csv(url,parse_dates=['Fecha'])

# Pasamos a % en lugar de sobre 1
valores = ['BBVA', 'Iberdrola', 'Inditex', 'Repsol', 'Santander']
df_bolsa[valores] = df_bolsa[valores]*100

# nos quedamos solo con los datos que sean números (asumimos que hemos comprobado que todos son ratio o intervalo)
df_bolsa_num = df_bolsa.select_dtypes(include=["number"])
normales(df_bolsa_num)

<a name="Vodka"></a>
### Test de igualdad de la media

Supongamos que tenemos 2 monedas, en la primera nos salen 2 caras seguidas, en la segunda 2 cruces seguidas ¿podemos asegurar que son monedas diferentes? Casi todos estaremos de acuerdo en que no, en que la diferencia se puede deber al azar. Eso sí, si tiramos cada moneda 1000 veces y en la primera sale por ejemplo un 30% más de caras que en la segunda puede que sospechemos que sí lo sean. Estas ideas son las que implementan los tests que comparan las medias de dos variables. Vamos a ver 2:

t de student: solo aplicable bajo ciertas circunstancias (es un test paramétrico): normalidad de ambas variables, misma varianza...la hipótesis nula (que rechazaremos si p<0.05) es que la media es la misma

Kolmogorov-Sminov: se puede aplicar a cualquier pareja de variables (pero es más exigente). La hipótesis nula (que rechazaremos si p<0.05) es que los datos provienen de la misma distribución continua

*t de Student*



0 significa cara, 1 significa cruz, tiramos la moneda 15 veces

In [ ]:
from scipy.stats import ttest_ind
x1 =  [1,0,1]*5
x2 = [0,1,0]*5
print(x1)
print(x2)
ttest_ind(x1, x2)

No podemos rechazar la posibilidad de que las dos monedas tengan la misma media.Probamos con 50 veces...

In [ ]:
from scipy.stats import ttest_ind
x1 =  [1,0,1]*25
x2 = [0,1,0]*25
print(x1)
print(x2)
ttest_ind(x1, x2)

números aleatorios siguiendo una normal 0,1

In [ ]:
from scipy.stats import ttest_ind
x1 =  np.random.randn(100000)
x2 = np.random.randn(100000)
ttest_ind(x1, x2)

como pvalue>0,05 no podemos rechazar la hipótesis de que la media es muy similar

In [ ]:
from scipy.stats import ttest_ind
x1 = np.random.randn(100000)+1
x2 = np.random.randn(100000)
ttest_ind(x1, x2)

Podemos rechazar que ambas medias sean iguales.

*Kolmogorov-Smirnov*

Realmente este test no busca comparar medidas sino ver si dos muestras proceden de la misma población. Veamos un ejemplo

In [ ]:
import pandas as pd
url = r"https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/madrid/Cont_Meteo_Traf.csv"
df_data =  pd.read_csv(url,parse_dates=['FECHAH'])
df_data["year"] = df_data.FECHAH.dt.year
df_data.columns

In [ ]:
df_data.year.value_counts()

In [ ]:
years = df_data.year.unique()
years = np.sort(years)
# un array de dataframes, uno por año
df_data_year = [df_data[df_data.year==y]["TEMPERATURA"] for y in years ]

In [ ]:
for y in years[1:]:
    t = df_data[df_data.year==y]["TEMPERATURA"]
    print(y,normal(t))

No son normales, vamos a utilizar el test de kolmogorov-smirnov para ver si la temperatura difiere de forma significativa en distintos años.

In [ ]:
for y1 in years[1:]:
    print("-----")
    for y2 in years[1:]:
      print(y1,y2)

In [ ]:
for y1 in years[1:]:
    print("-----")
    for y2 in years[1:]:
      if y2>y1:
        print("*",end="")
      print(y1,y2)

In [ ]:
from scipy.stats import kstest

for y1 in years[1:]:

    for y2 in years[1:]:
        if y2>y1:
            t1 = df_data[df_data.year==y1]["TEMPERATURA"]
            t2 = df_data[df_data.year==y2]["TEMPERATURA"]
            print(y1,y2,kstest(t1, t2),round(t1.mean(),2),round(t2.mean(),2))


Todos los años corresponden a temperaturas diferentes con una diferencia estadísticamente signficativa

<a name="Bootstrapping"></a>
### Bootstrapping

El último método, y seguramente el más efectivo, se basa en una aplicaicón del llamado "Teorema central del límite". Este teorema aplicado a nuestro contexto dice que, si se toman distintas muestras de una misma población, y se calcula un cierto estadístico (media, mediana, otro) sobre cada muestra, el conjunto de valores que obtenemos sigue una distribución normal.

Lo bueno de esta propuesta es que al quedar una normal el conjunto de todas las medidas o experimentos, podemos aplicar la  la llamada _regla empírica_

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/22/Empirical_rule_histogram.svg/1280px-Empirical_rule_histogram.svg.png" width=400> </img>

Es decir, que podemos considerar que el estadístico viene representado por la media de los experimentos y además tenemos, de paso, un intervalo de confianza.

Ejemplo: queremos comparar la media de los valores del BBVA Jueves y Viernes

In [ ]:
filtroJ = df_bolsa["Día"] == "Jueves"
filtroV = df_bolsa["Día"] == "Viernes"
filtroL = df_bolsa["Día"] == "Lunes"

df_bolsa[filtroJ].BBVA.mean(), df_bolsa[filtroV].BBVA.mean(), df_bolsa[filtroL].BBVA.mean()

¿Es significativa este diferencia?

In [ ]:
from tqdm import tqdm

sJ = []
sV = []
sL = []
tamaño = 50 # por poner algo, que sea una cantidad menor que los dataframes correspondientes
reps = 20000
for i in tqdm(range(reps)):
    sJ.append( df_bolsa[filtroJ].BBVA.sample(tamaño).mean())
    sV.append( df_bolsa[filtroV].BBVA.sample(tamaño).mean())
    sL.append( df_bolsa[filtroL].BBVA.sample(tamaño).mean())

df_medias = pd.DataFrame({"Lunes":sL, "Jueves":sJ, "Viernes":sV})
df_medias

In [ ]:
df_medias.Jueves.plot(kind="hist",bins=50)

In [ ]:
for c in df_medias:
  print(c,df_medias[c].skew(), df_medias[c].kurtosis())

Ahora utilizamos la "ley empírica" para obtener los intervalos:

In [ ]:
muL,muJ, muV = df_medias.Lunes.mean(), df_medias.Jueves.mean(), df_medias.Viernes.mean()
sigmaL,sigmaJ, sigmaV = df_medias.Lunes.std(),df_medias.Jueves.std(), df_medias.Viernes.std()

print(f"Lunes : {100*muL:.4} [{100*(muL-2*sigmaL):.4f},{100*(muL+2*sigmaL):.4f}]")
print(f"Jueves : {100*muJ:.4} [{100*(muJ-2*sigmaJ):.4f},{100*(muJ+2*sigmaJ):.4f}]")
print(f"Viernes: {100*muV:.4} [{100*(muV-2*sigmaV):.4f},{100*(muV+2*sigmaV):.4f}]")

Hay solapamiento, de momento no podemos asegurar que las medias sean diferentes.

La implementación en librerías tiene ventajas:


- Utiliza muestras con reemplazamiento
- Por tanto es adecuado para dataframes con pocos datos
- Igualmente obtenemos intervalos de confianza
- Utiliza técnicas más refinadas que la regla empírica y logra intervalos más pequeños

In [ ]:
import pandas as pd
from scipy.stats import bootstrap

filtro = df_bolsa["Día"] == "Jueves"
# columna sin nulos
bbva = df_bolsa[filtro]["BBVA"].dropna()

res = bootstrap((bbva,), np.mean, confidence_level=0.95,n_resamples=10000, method='percentile')
ci = res.confidence_interval

print(f"Jueves Media: { ci.low + (ci.high-ci.low)/2:.4f} [{ci.low:.4f},{ci.high:.4f}]")

In [ ]:
def ic(df,columnFiltro,columnExaminar,value):
  filtro = df[columnFiltro] == value
  # columna sin nulos
  col = df[filtro][columnExaminar].dropna()

  res = bootstrap((col,), np.mean, confidence_level=0.95,n_resamples=30000, method='percentile')
  ci = res.confidence_interval

  print(f"Media de {columnExaminar} con {columnFiltro}=={value}: { ci.low + (ci.high-ci.low)/2:.4f}. CI: [{ci.low:.4f},{ci.high:.4f}]")
  return res.confidence_interval


ic(df_bolsa,"Día","BBVA","Lunes")
ic(df_bolsa,"Día","BBVA","Martes")
ic(df_bolsa,"Día","BBVA","Miércoles")
ic(df_bolsa,"Día","BBVA","Jueves")
ic(df_bolsa,"Día","BBVA","Viernes")
print("="*50)

Otra forma más precisa aun: concentrarse directamente en la diferencia de medias

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import bootstrap

def mean_diff(data1, data2):
    return (data1.mean() - data2.mean())

url = "https://github.com/RafaelCaballero/tdm/raw/refs/heads/master/datos/IBEX6000Inc.zip"
# solución
df_raw = pd.read_csv(url,parse_dates=['Fecha']).dropna()

# las dos columnas
filtro = df_raw["Día"] == "Lunes"
lunes = df_raw[filtro]["BBVA"]*100
filtro = df_raw["Día"] == "Martes"
martes = df_raw[filtro]["BBVA"]*100

# Bootstrapping para calcular la diferencia de medias
boot_result = bootstrap((lunes, martes), mean_diff,  confidence_level=0.95)

# Resultados
print(f"Diferencia de medias: {mean_diff(lunes, martes):.4f}")
print(f"Intervalo de confianza al 95%: [{boot_result.confidence_interval.low:.4f},{boot_result.confidence_interval.high:.4f}]")

Como el intervalo no incluye al 0 la media no puede ser 0, así que son diferentes medias de forma estadísticamente significativa

<a name="Referencias"></a>
### Referencias

[Medidas de centralidad](https://statistics.laerd.com/statistical-guides/measures-central-tendency-mean-mode-median.php)<br>
[Biblioteca ydata-profiling](https://pypi.org/project/ydata-profiling/)<br>

[Notebook](https://github.com/mGalarnyk/Python_Tutorials/blob/master/Statistics/boxplot/Box_plot_interpretation.ipynb) con ejemplos de Boxplot incluyendo variantes como el "notch"

*Data Cleaning*. Ihab F. Ilyas and Xu Chu. Association for Computing Machinery 9781450371544 Tiene un capítulo dedicado a outliers muy completo


[Best Practices in Data Cleaning: A Complete Guide to Everything You Need to Do Before and After Collecting Your Data](http://pzs.dstu.dp.ua/DataMining/preprocessing/bibl/cleaning.pdf). Capítulo 7.
de Jason W. Osborne. Un poco técnico/estadístico pero muy preciso,

[Outliers multivariante](https://medium.com/analytics-vidhya/anomaly-detection-in-python-part-1-basics-code-and-standard-algorithms-37d022cdbcff) Artículo de donde he tomado el código y el ejemplo para este apartado

[Outliers scikit-learn](https://scikit-learn.org/1.5/modules/outlier_detection.html) Ouliers con la biblioteca estándar de machine-learning. Quizás un poco complejo, pero muy potente.

[Cómo calcular el número óptimo de bins en un histograma](https://medium.com/@maxmarkovvision/optimal-number-of-bins-for-histograms-3d7c48086fde)

[Teorema central del límite](https://www.geeksforgeeks.org/central-limit-theorem/?ref=oin_asr1) buena explicación con muchos ejemplos.

[Cálculo de outliers en datos asimétricos](https://wis.kuleuven.be/stat/robust/papers/2008/outlierdetectionskeweddata-revision.pdf). Artículo (en inglés) que define outliers a partir de la medida [medcouple](https://en.wikipedia.org/wiki/Medcouple) una medida de asimetría robusta frente a outliers<br>

[Bootstrapping](https://cienciadedatos.net/documentos/pystats04-bootstrapping-python) bien explicado, paso a paso<br>
